# 🏦 Magentic Multi-Agent Orchestration for Investment Research

## Overview

This notebook demonstrates **Magentic orchestration** for financial services - an advanced multi-agent coordination pattern for comprehensive investment research and analysis. The orchestrator automatically manages research task decomposition, specialist agent selection, and insight synthesis.

> ⚠️ **Financial Services Disclaimer**: This notebook demonstrates AI agent workflows for educational purposes. In production, all investment research and recommendations must comply with regulatory requirements (SEC, FINRA) and include appropriate disclosures.

### Industry Use Case: Investment Research Report

A wealth management firm needs to prepare quarterly investment research reports that require:
- **Market Research**: Current market conditions, sector trends
- **Quantitative Analysis**: Financial calculations, risk metrics
- **Report Synthesis**: Consolidated insights and recommendations

### Key Concepts

| Concept | Description |
|---------|-------------|
| **MagenticBuilder** | High-level API for multi-agent orchestration |
| **Standard Manager** | Built-in orchestrator with planning capabilities |
| **Specialized Agents** | Domain experts (Market Researcher, Quant Analyst) |
| **Streaming Callbacks** | Real-time event monitoring |
| **Event Types** | Orchestrator messages, agent deltas, agent messages, final result |

### Architecture

```
Investment Research Request
           ↓
Magentic Orchestrator (Standard Manager)
           ↓
Research Plan Generation
           ↓
┌─────────────────────────────────┐
│     Delegate to Specialists     │
│  ├── MarketResearcherAgent      │
│  │   (Market data, sector news) │
│  └── QuantAnalystAgent          │
│      (Financial calculations)   │
└─────────────────────────────────┘
           ↓
Monitor & Adapt Plan
           ↓
Synthesize Investment Report
```

### Workflow Parameters

| Parameter | Default | Purpose |
|-----------|---------|---------|
| `max_round_count` | 10 | Maximum orchestration rounds |
| `max_stall_count` | 3 | Retries when progress stalls |
| `max_reset_count` | 2 | Full plan resets allowed |

## Prerequisites

- OpenAI API key configured: `OPENAI_API_KEY` environment variable
- Agent Framework installed: `pip install agent-framework`
- Special models for some agents:
  - MarketResearcherAgent: `gpt-4o-search-preview` (web search capability)
  - QuantAnalystAgent: OpenAI Assistants with code interpreter

## 1️⃣ Setup and Imports

In [ ]:
import asyncio
import os
from dotenv import load_dotenv
from typing import cast

from azure.identity import AzureCliCredential

from agent_framework import (
    AgentRunUpdateEvent,
    ChatAgent,
    ChatMessage,
    MagenticBuilder,
    WorkflowOutputEvent,
)
from agent_framework.azure import AzureOpenAIChatClient

# Load environment variables from .env file
load_dotenv('../../.env')

# Verify environment is loaded
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
print(f"✅ Environment loaded: {azure_endpoint is not None and deployment_name is not None}")

## 2️⃣ Create Azure OpenAI Chat Client

We'll use a single Azure OpenAI client for all agents to share credentials and endpoint configuration.

In [ ]:
# Create shared Azure OpenAI Chat Client
chat_client = AzureOpenAIChatClient(
    credential=AzureCliCredential(),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
)

print("✅ Azure OpenAI Chat Client created")

## 3️⃣ Create Specialized Financial Agents

### MarketResearcherAgent
- Specializes in market data, sector trends, and financial news
- Gathers factual information about market conditions

### QuantAnalystAgent
- Performs financial calculations and risk analysis
- Provides detailed analysis with risk metrics and projections

In [ ]:
# Create Market Researcher Agent
market_researcher = ChatAgent(
    name="MarketResearcherAgent",
    description="Specialist in market research, sector analysis, and financial news gathering",
    instructions=(
        "You are a Financial Market Researcher at a wealth management firm. "
        "You gather market data, sector trends, company news, and economic indicators. "
        "Focus on factual information without performing calculations. "
        "Always cite sources and note the date of any market data you reference."
    ),
    chat_client=chat_client,
)

# Create Quantitative Analyst Agent
quant_analyst = ChatAgent(
    name="QuantAnalystAgent",
    description="A quantitative analyst that performs financial calculations, risk analysis, and portfolio modeling.",
    instructions=(
        "You are a Quantitative Analyst at a wealth management firm. "
        "You perform financial calculations including: portfolio analysis, risk metrics (beta, Sharpe ratio, VaR), "
        "return calculations, and investment projections. "
        "Always show your calculation methodology and assumptions. "
        "Present results in clear tables when appropriate."
    ),
    chat_client=chat_client,
)

# Create Manager Agent to orchestrate the workflow
manager_agent = ChatAgent(
    name="ResearchManager",
    description="Orchestrator that coordinates the investment research workflow",
    instructions=(
        "You coordinate a team of financial analysts to complete investment research tasks efficiently. "
        "Ensure all research includes appropriate risk disclosures."
    ),
    chat_client=chat_client,
)

print("✅ Market Researcher Agent created")
print("✅ Quant Analyst Agent created")
print("✅ Manager Agent created")

## 4️⃣ Build Magentic Investment Research Workflow

### Configuration

| Method | Purpose |
|--------|---------|
| `.participants(...)` | Define available specialist agents |
| `.with_standard_manager(...)` | Configure orchestrator with Azure OpenAI |

In [ ]:
print("\n🏦 Building Investment Research Workflow...")

workflow = (
    MagenticBuilder()
    .participants([market_researcher, quant_analyst])
    .with_standard_manager(
        agent=manager_agent,
        max_round_count=10,
        max_stall_count=3,
        max_reset_count=2,
    )
    .build()
)

print("✅ Magentic workflow built successfully")

## 5️⃣ Define Investment Research Task

This quarterly research request requires:

1. **Market Research**: Current technology sector trends and market conditions
2. **Quantitative Analysis**: Calculate risk-adjusted returns and portfolio metrics
3. **Comparison**: Evaluate different investment approaches
4. **Synthesis**: Provide actionable recommendations with risk assessment

The orchestrator will automatically decompose this into subtasks and delegate to the appropriate specialists.

In [ ]:
# Investment research request from a wealth advisor
research_request = (
    "I need a quarterly investment research brief for a client considering technology sector exposure. "
    "Please research the current state of the technology sector including recent performance of major indices "
    "(NASDAQ-100, S&P Technology Select Sector) and key market drivers. "
    "Then calculate the risk-adjusted return metrics for a hypothetical $100,000 investment allocated as: "
    "40% large-cap tech ETF (assume 12% annual return, 18% volatility), "
    "35% mid-cap growth fund (assume 15% annual return, 25% volatility), "
    "and 25% dividend tech stocks (assume 8% annual return, 12% volatility). "
    "Provide a summary table comparing expected returns, portfolio volatility, and Sharpe ratio "
    "(assuming 5% risk-free rate). Include a recommendation with risk considerations."
)

print(f"📋 Research Request: {research_request}")

## 6️⃣ Execute Research Workflow with Streaming

The workflow execution:
1. Generates research plan (task decomposition)
2. Delegates to MarketResearcher for sector data
3. Delegates to QuantAnalyst for calculations
4. Monitors progress and adapts if needed
5. Synthesizes final research report
6. Completes when idle or max rounds reached

In [ ]:
async def run_investment_research():
    """Run the Magentic investment research workflow with streaming output."""
    
    print("\n🚀 Starting investment research workflow...")
    print("=" * 60)
    
    output_event: WorkflowOutputEvent | None = None
    last_message_id: str | None = None
    
    async for event in workflow.run_stream(research_request):
        if isinstance(event, AgentRunUpdateEvent):
            message_id = event.data.message_id
            if message_id != last_message_id:
                if last_message_id is not None:
                    print("\n")
                print(f"- {event.executor_id}:", end=" ", flush=True)
                last_message_id = message_id
            print(event.data, end="", flush=True)
        
        elif isinstance(event, WorkflowOutputEvent):
            output_event = event
    
    # Display final output
    print("\n\n" + "=" * 60)
    print("📊 INVESTMENT RESEARCH REPORT COMPLETE")
    print("=" * 60)
    
    if output_event:
        output_messages = cast(list[ChatMessage], output_event.data)
        if output_messages:
            output = output_messages[-1].text
            print(output)
    
    print("\n" + "=" * 60)
    print("⚠️ DISCLAIMER: This is for demonstration purposes only.")
    print("   Actual investment decisions require professional advice.")

## 7️⃣ Run the Investment Research Workflow

In [ ]:
await run_investment_research()

## 📝 Key Takeaways

### Magentic for Financial Research

| Benefit | Description |
|---------|-------------|
| **Intelligent Planning** | Orchestrator decomposes complex research requests |
| **Specialist Delegation** | Market researcher vs. quantitative analyst |
| **Adaptive Execution** | Adjusts plan based on findings |
| **Synthesized Reports** | Consolidated insights from multiple specialists |

### Industry Use Cases for Magentic Orchestration

| Use Case | Specialists Needed |
|----------|-------------------|
| Investment Research | Market Researcher + Quant Analyst |
| Due Diligence | Legal Analyst + Financial Analyst |
| Risk Assessment | Risk Analyst + Market Researcher |
| Client Reporting | Data Analyst + Compliance Writer |

### Magentic vs. Other Orchestration Patterns

| Pattern | When to Use | Key Feature |
|---------|-------------|-------------|
| **Concurrent** | Independent parallel tasks | Fan-out/fan-in |
| **Sequential** | Linear, dependent steps | Shared context |
| **Magentic** | Complex, adaptive decomposition | Intelligent orchestration |


### Production Considerations for FSI

| Consideration | Recommendation |
|---------------|----------------|
| **Compliance** | Log all orchestrator decisions for audit trail |
| **Cost Control** | Set appropriate max_round_count |
| **Data Currency** | Timestamp all market data references |
| **Disclaimers** | Include regulatory disclosures in final output |
| **Review** | Human review required before client delivery |